#  任务4 模型选择(3天)

# 以lightGBM为例

In [2]:
#coding:utf-8
#导入warnings包，利用过滤器来实现忽略警告语句。
import warnings
warnings.filterwarnings('ignore')

# GBDT
from sklearn.ensemble import GradientBoostingRegressor
# XGBoost
import xgboost as xgb
# LightGBM
import lightgbm as lgb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
import pickle
import multiprocessing
from sklearn.preprocessing import StandardScaler
ss = StandardScaler() 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC,LinearRegression,LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" 

In [36]:
train_df = pd.read_csv('/home/cc/holdshy/XJQ/数据竞赛（房租预测）/train_data(featured).csv', index_col=0)
test_df = pd.read_csv('/home/cc/holdshy/XJQ/数据竞赛（房租预测）/test_a(cleared).csv', index_col=0)
# test = pd.read_csv('/home/cc/holdshy/XJQ/数据竞赛（房租预测）/test_a.csv')
# train_df.columns
# test.columns
# test_df.to_csv('/home/cc/holdshy/XJQ/数据竞赛（房租预测）/test_a(cleared).csv')

In [ ]:
# train_df.loc[:,'RoomDir']=LabelEncoder().fit_transform(train_df.loc[:,'RoomDir'])
# test_df.loc[:,'RoomDir']=LabelEncoder().fit_transform(test_df.loc[:,'RoomDir'])

X_train = train_df.drop(['tradeMoney'],axis=1)
Y_train = train_df.loc[:,'tradeMoney']
X_test = test_df.drop(['ID'],axis=1)
test_id = test_df.loc[:,'ID']

xgb_reg = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468,
                             learning_rate=0.05, max_depth=3,
                             min_child_weight=2, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=True,
                             random_state=7, nthread=-1)
xgb_reg.fit(X_train,Y_train)
Y_pred = xgb_reg.predict(X_test)

sub = pd.DataFrame({
    'id':test_id,
    'price':Y_pred
})
sub.to_csv('/home/cc/holdshy/XJQ/数据竞赛（房租预测）/baseline.csv',index=False)    #baseline：2.554


In [ ]:
def xgb_eval(train_df,val_df):
    train_df=train_df.copy()
    val_df=val_df.copy()

    try:
        from sklearn.preprocessing import LabelEncoder
        lb_encoder=LabelEncoder()
        lb_encoder.fit(train_df.loc[:,'RoomDir'].append(val_df.loc[:,'RoomDir']))
        train_df.loc[:,'RoomDir']=lb_encoder.transform(train_df.loc[:,'RoomDir'])
        val_df.loc[:,'RoomDir']=lb_encoder.transform(val_df.loc[:,'RoomDir'])
    except Exception as e:
        print(e)

    import xgboost as xgb
    X_train=train_df.drop(['Rental'],axis=1)
    Y_train=train_df.loc[:,'Rental'].values
    X_val=val_df.drop(['Rental'],axis=1)
    Y_val=val_df.loc[:,'Rental'].values

    from sklearn.metrics import mean_squared_error

    try:
        eval_df=val_df.copy().drop('Time',axis=1)
    except Exception as e:
        eval_df=val_df.copy()

    reg_model=xgb.XGBRegressor(max_depth=5,n_estimators=500,n_jobs=-1)
    reg_model.fit(X_train,Y_train)

    y_pred=reg_model.predict(X_val)
    print(np.sqrt(mean_squared_error(Y_val,y_pred)),end=' ')

    eval_df.loc[:,'Y_pred']=y_pred
    eval_df.loc[:,'RE']=eval_df.loc[:,'Y_pred']-eval_df.loc[:,'Rental']
    
    print('')
    feature=X_train.columns
    fe_im=reg_model.feature_importances_
    print(pd.DataFrame({'fe':feature,'im':fe_im}).sort_values(by='im',ascending=False))

    import matplotlib.pyplot as plt
    plt.clf()
    plt.figure(figsize=(15,4))
    plt.plot([Y_train.min(),Y_train.max()],[0,0],color='red')
    plt.scatter(x=eval_df.loc[:,'Rental'],y=eval_df.loc[:,'RE'])
    plt.show()

    return eval_df
